<a href="https://colab.research.google.com/github/lorenzlorg/MSc-Thesis-CV/blob/main/2_GEOREF_STEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GEOREFERENCING STEP**

Questo notebook serve per georeferenziare le detection ottenute con il notebook precedente. 

# Import e funzioni

In [ ]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 64.3 MB/s eta 0:00:00


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.6 MB/s eta 0:00:00


In [ ]:
import cv2
import time
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.neighbors import BallTree

In [ ]:
#HELPER FUNCTION TO GET CORRESPONDENCES BETWEEN TWO ARRAYS
def find_rows(a, b):
    """
    https://stackoverflow.com/questions/20230384/find-indexes-of-matching-rows-in-two-2-d-arrays
    :param a:
    :param b:
    :return:
    """
    dt = np.dtype((np.void, a.dtype.itemsize * a.shape[1]))

    a_view = np.ascontiguousarray(a).view(dt).ravel()
    b_view = np.ascontiguousarray(b).view(dt).ravel()

    sort_b = np.argsort(b_view)
    where_in_b = np.searchsorted(b_view, a_view, sorter=sort_b)
    where_in_b = np.take(sort_b, where_in_b)
    which_in_a = np.take(b_view, where_in_b) == a_view
    where_in_b = where_in_b[which_in_a]
    which_in_a = np.nonzero(which_in_a)[0]

    return np.column_stack((which_in_a, where_in_b))


#HELPER FUNCTION TO CONVERT PIXELS IN COORDINATES
def get_coords(pixel):
    return src_geo.xy(pixel[1], pixel[0])


#HELPER FUNCTION TO FIND NEAREST POINT BETWEEN MATCHED AND NOT MATCHED
def get_nearest(src_points, candidates, dist_metric, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=100, metric=dist_metric)
    # haversine
    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)

# Lettura immagini e file detection

In [ ]:
# immagine originale, rettangolare con rumore
img = cv2.imread('/content/Grid_Bocconi.png')  # img random
# https://drive.google.com/file/d/1x2K7HHnQPzqx6p7ok7--Qm45QqHXw2uk/view?usp=share_link


# immagine disorta
# src = cv2.imread('/content/Grid_Bocconi_modificatoo.tif')  # img random distorta - si ottiene caricando i CGP su Grid_Bocconi.png e georeferenziando
# src = cv2.imread('/content/Grid_Bocconi_modificato.tif')  # img random distorta
src = cv2.imread('/content/Grid_Bocconi_modified V3.tif')  # img random distorta
# https://drive.google.com/file/d/1W3F_85F8eNCME8_hGWS10ViGhVw6GR4R/view?usp=share_link



# da usare alla fine, è l'immagine random distorta contenente informazioni geospaziali
# src_geo = rasterio.open('/content/Grid_Bocconi_modificatoo.tif')  # img random distorta x2
# src_geo = rasterio.open('/content/Grid_Bocconi_modificato.tif')  # img random distorta x2
src_geo = rasterio.open('/content/Grid_Bocconi_modified V3.tif')  # img random distorta x2

In [ ]:
# file txt contenente le detection
detections = np.loadtxt('/content/drive/MyDrive/detections_YOLOv7_SORT/milan_2021-07-15_14_59_31/Copia di detection_milan_2021-07-15_14_59_31_ordered.txt')  # detection ottenute con yolo+sort

detections = detections[~np.any(detections[:,2].reshape(len(detections),1) == 1080, axis=1)]  # TEMPORANEO: filtro le detections in cui sono presenti 1080 (out of range) - da controllare creazione mapping detections

# 1. trovo le corrispondenze tra i valori rgb dell'immagine distorta e di quella originale (a livello di pixel)

dato che i colori nelle due immagini sono univoci, quindi posso trovare una relazione 1:1 per tutti i colori da immagine originale a immagine distorta.

Nell'immagine originale aggiungo una riga di nero in modo tale che il nero nell'immagine tiff sia matchabile

In [ ]:
img.shape

(1080, 1920, 3)

In [ ]:
src.shape

(1015, 2340, 3)

In [ ]:
# corrispondenza tra colori nelle due immagini dato che sono univoci

img_r, src_r = img.flatten(), src.flatten()

In [ ]:
img_r.shape

(6220800,)

In [ ]:
src_r.shape

(7125300,)

In [ ]:
img_r, src_r = img.reshape((int(len(img_r)/3), 3)), src.reshape((int(len(src_r)/3), 3))  # reshape immagine per avere lista univoca di colori

In [ ]:
img_r.shape

(2073600, 3)

In [ ]:
src_r.shape

(2375100, 3)

In [ ]:
black = np.array([0,0,0]).astype(int)
img_r_black = np.vstack((img_r,black)).astype('uint8')  # aggiungo il nero in fondo per avere tutte le corrispondenze, il nero sarà in tutte quelle zone che si sono venute a creare con la distorsione dell'immagine

# "aggiungo il nero in fondo per avere tutte le corrispondenze" nel senso che nell'immagine tiff abbiamo del nero nello sfondo però nell'immagine orginale rettangolare non abbiamo il nero. Se aggiungiamo uan riga di nero cosi riesco a fare il match, cioè i pixel neri delle due immagini si matcheranno

In [ ]:
img_r_black.shape

(2073601, 3)

In [ ]:
src_r.shape

(2375100, 3)

In [ ]:
colors_geo_to_origin = find_rows(src_r, img_r_black)  # corrispondenza tra indice del colore nell'immagine distorta e indice nell'immagine originale, dato che avrò due matrici con elementi i valori rgb dei vari pixel che sono tutti diversi tra loro. Noi vogliamo trovare il matching tra una matrice e l'altra, quindi tra l'immagine distorta e quella non distorta
# RISULTATO: array con numero corrispondenze = numero pixel dell'immagine distorta
# dal momento in cui l'immagine è distorta alcuni colori che ci sono in un'immagine non esistono nell'altra o alcuni colori si presentano più di una volta -> quindi l'algo fa si che ci sia un match per i pixel che esistono in un'immagine e nell'altra no usando la tecninca del vicino più prossimo e per i pixel che si presentano più di una volta si prendono quelli con la miglior location
print('1:ok')

1:ok


In [ ]:
src_r.shape

(2375100, 3)

In [ ]:
img_r_black.shape

(2073601, 3)

In [ ]:
colors_geo_to_origin  # ma img_r_black è più piccola, come si fa ad avere un match sulla dimensione più grande

# 2073600 guardando sotto forse si riferisce al nero
# Generate a 1920 x 1080 graphic with 2,073,600 unique colours
# quindi 2073600 è il nero

array([[      0, 2073600],
       [      1, 2073600],
       [      2, 2073600],
       ...,
       [2375097, 2073600],
       [2375098, 2073600],
       [2375099, 2073600]])

In [ ]:
colors_geo_to_origin  # prima colonna immagine distorta, questi pixel neri vengono mappati con l'ultima riga nera ggiunta di proposito nell'immagine originale rettangolare

# corrispondenza pixel img originale e pixel img distorta

# corrispondenza tra pixel, non siamo ancora a livello di coordinate reali

array([[      0, 2073600],
       [      1, 2073600],
       [      2, 2073600],
       ...,
       [2375097, 2073600],
       [2375098, 2073600],
       [2375099, 2073600]])

# 2. combino i pixel residui (senza corrispondenza) dell'immagine originale con quelli che hanno invece una corrispondenza nell'immagine distorta


nell’immagine distorta vado a perdere delle aree quindi dei pixel dell’immagine originale non hanno una corrispondenza, li associo a quei pixel vicini che hanno una corrispondenza

In [ ]:
px, py = np.meshgrid(np.arange(img.shape[1]), np.arange(img.shape[0]))
px, py = px.flatten(), py.flatten()
i_array = np.vstack((px, py)).T  # creo un array 2D con gli indici dei pixel che corrispondono all'immagine originale

img_rgb_idx = np.hstack((img_r, i_array))  # combino colori e indici nell'immagine originale
index_img = np.arange(len(img_rgb_idx))
img_rgb_idx = np.hstack((img_rgb_idx, index_img.reshape(len(img_rgb_idx),1)))

matched_origin = colors_geo_to_origin[~np.any(colors_geo_to_origin == 2073600, axis=1)] # elimino indice aggiunto per mappare il nero
matched_origin_unique = np.unique(matched_origin[:,1])  # indici originali non matchati - dovrebbero essere i pixel residui
not_matched_origin = np.setdiff1d(index_img, matched_origin_unique)

matched, not_matched = img_rgb_idx[matched_origin_unique], img_rgb_idx[not_matched_origin]
matched_idx_or, not_matched_idx_or = matched[:,[3,4]], not_matched[:,[3,4]]

nearest_points, nearest_dist = get_nearest(not_matched_idx_or, matched_idx_or, 'minkowski')  # per i punti non matchati prendo come corrispondenza il match di un punto vicino
new_match = matched[nearest_points]  # indice delle coordinate più vicine al punto originale dove non c'è corrispondenza diretta
sorted_unique, index_unique, inverse_unique, counts_unique = np.unique(nearest_points, axis=0, return_index=True, return_inverse=True, return_counts=True)
all_counts = counts_unique[inverse_unique]
print('2:ok')

2:ok


# 3. cambio il valore del pixel delle detection  (file txt delle detection di YOLO) dove non c'è corrispondenza tra immagine originale e distorta con valore nuovo sulla base di quanto trovato precedentemente

ora prendendo il file delle detection, se una detection si trova in un punto che non ha corrispondenza nell’immagine distorta allora cambio il valore dei suoi pixel sulla base dei risultati precedentei. Questo perchè le detection sono su un frame la cui dimensione è esattamente quella dell'immagine originale con rumore. Quindi ogni punto sul frame ha il suo corrispondente sull'immagine originale. Quindi se una detection si trova in un punto che nell'immagine originale non ha la sua corrispondenza nell'immagine distorta allora questa detection non potrà essere georeferenziata.

Infatti prima se un un pixel (dell'immagine originale con rumore) non aveva corrispondenza allora si faceva matchare con la corrispondenza del pixel più vicino a lui.

Detto ciò dobbiamo cambiare le coordinate delle detection che ci sono in quei punti che non hanno corrispondenza.

In [ ]:
detections[:, [2, 3]] # seleziono colonne coordinate x e y

array([[1647. ,  950. ],
       [1280. ,  987. ],
       [1264. ,  986. ],
       ...,
       [1160. ,  657. ],
       [1498. ,  692. ],
       [ 781.5,  762. ]])

In [ ]:
detections[:,[0,1,4]]

array([[1.00000e+00, 0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.00000e+00, 0.00000e+00],
       [1.00000e+00, 2.00000e+00, 0.00000e+00],
       ...,
       [2.78900e+04, 2.58103e+05, 0.00000e+00],
       [2.78900e+04, 2.58104e+05, 0.00000e+00],
       [2.78900e+04, 2.58105e+05, 0.00000e+00]])

In [ ]:
det_points = detections[:, [2, 3]].astype(int)
dest_near_idx, dest_near_dist = get_nearest(det_points, matched_idx_or, 'minkowski')
dets_matched = matched_idx_or[dest_near_idx]
detections = np.hstack((detections[:,[0,1,4]],dets_matched)).astype(int)  # metto come prime 3 colonne le restanti, quindi quelle diverse dalla colonne indicanti le coordinate
detections

# ora il file detection è stato aggiornato e i punti delle detection che non avrebbero avuto una corrispondenza ora la avranno grazie alla logica del punto vicino che ha invece corrispondenza

array([[     1,      0,      0,   1647,    949],
       [     1,      1,      0,   1279,    988],
       [     1,      2,      0,   1262,    987],
       ...,
       [ 27890, 258103,      0,   1160,    657],
       [ 27890, 258104,      0,   1497,    692],
       [ 27890, 258105,      0,    781,    762]])

In [ ]:
# ora tutti. i punti su cui sorgono le detection sono mappati, nel caso in cui originariamente non avevano una mappatura verranno ora mappati sul vicino che ha invece una corrispondenza

# 4. trovo la corrispondenza tra detection e immagine originale



In [ ]:
# e trovo i famosi puntini rossi presenti sull'immagine originale piena di rumore

In [ ]:
dets_matched

array([[1647,  949],
       [1279,  988],
       [1262,  987],
       ...,
       [1160,  657],
       [1497,  692],
       [ 781,  762]])

In [ ]:
i_array

array([[   0,    0],
       [   1,    0],
       [   2,    0],
       ...,
       [1917, 1079],
       [1918, 1079],
       [1919, 1079]])

In [ ]:
index_dets_to_origin = find_rows(dets_matched, i_array)  # trovo la corrispondenza di ogni detection con l'indice originale per poter ritrasporre in 3d
# RISULTATO: array con numero corrispondenze = numero di detections
print('4:ok')

4:ok


In [ ]:
index_dets_to_origin

array([[      0, 1823727],
       [      1, 1898239],
       [      2, 1896302],
       ...,
       [9446272, 1262600],
       [9446273, 1330137],
       [9446274, 1463821]])

# 5. trovo la corrispondenza tra le detections nell'immagine originale e il loro indice nell'immagine distorta

una volta che tutti i punti dell’immagine originale sono mappati nell’immagine distorta

dunque ora vogliamo che i puntini rossi siano visibili anche sull'immagine disorta piena di rumore

In [ ]:
index_dets_to_origin[:,1]

array([1823727, 1898239, 1896302, ..., 1262600, 1330137, 1463821])

In [ ]:
colors_geo_to_origin[:,1]

array([2073600, 2073600, 2073600, ..., 2073600, 2073600, 2073600])

In [ ]:
# prendo i pixel nell'immagine originale che sono stati matchati con le detection e li matcho ora con l'immagine distorta

index_dets_to_geo = find_rows(index_dets_to_origin[:,1].reshape((len(index_dets_to_origin),1)), colors_geo_to_origin[:,1].reshape(len(colors_geo_to_origin),1))
# RISULTATO: corrispondenza tra indice delle detection nell'immagine originale e indice delle detection nell'immagine distorta
# -> visto che c'è perfetta corrispondenza tra indice dell'immagine distorta e indice array, il valore [:,1] corrisponde immediatamente all'immagine distorta
print('5:ok')

5:ok


In [ ]:
index_dets_to_geo

array([[      0, 1685027],
       [      1, 1488458],
       [      2, 1481439],
       ...,
       [9446272, 1678560],
       [9446273, 1938165],
       [9446274, 1228972]])

# 6. trovo la corrispondenza tra l'indice nell'immagine distorta e la coordinata geografica

qui entra in gioco l’immagine distorta con metadati geografici



In [ ]:
sx, sy = np.meshgrid(np.arange(src.shape[1]), np.arange(src.shape[0]))
sx, sy = sx.flatten(), sy.flatten()
s_array = np.vstack((sx, sy)).T

dets_in_dist = s_array[index_dets_to_geo[:,1]]
dets_in_geo = np.apply_along_axis(get_coords, 1, dets_in_dist)

frame_idx = detections[index_dets_to_geo[:,0]] # filtro l'indice delle immagini
dets_in_geo = np.hstack((frame_idx[:,[0,1,2]],dets_in_geo))

df = pd.DataFrame(dets_in_geo[:, [0,1,2]].astype(int), columns=['frame','id','class'])

geo_dets = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(dets_in_geo[:, 3], dets_in_geo[:, 4]), crs='EPSG:32632')
geo_dets.to_file('detection_milan_2021-07-15_14_59_31.geojson', driver='GeoJSON')
print('6:ok')

6:ok


In [ ]:
geo_dets.head()

,frame,id,class,geometry
0,1,0,0,POINT (514782.468 5034508.365)
1,1,1,0,POINT (514780.687 5034524.989)
2,1,2,0,POINT (514780.885 5034525.582)
3,1,3,0,POINT (514798.300 5034522.020)
4,1,4,0,POINT (514776.531 5034562.392)


In [ ]:
!cp /content/detection_milan_2021-07-15_14_59_31.geojson /content/drive/MyDrive

in questo modo dovrei aver georeferenziato tutte le mie detection, basta che trascino il file .geojson su qgis con la mappa aperta e tutto!

# 7. arricchimento dataset

In [ ]:
# dataset ottenuto
geo_dets.head()

# ora voglio aggiungere un campo data con l'orario relativo al singolo frame

,frame,id,class,geometry
0,1,0,0,POINT (514782.468 5034508.365)
1,1,1,0,POINT (514780.687 5034524.989)
2,1,2,0,POINT (514780.885 5034525.582)
3,1,3,0,POINT (514798.300 5034522.020)
4,1,4,0,POINT (514776.531 5034562.392)


In [ ]:
geo_dets
# numero frame: 1979

,frame,id,class,geometry
0,1,0,0,POINT (514782.468 5034508.365)
1,1,1,0,POINT (514780.687 5034524.989)
2,1,2,0,POINT (514780.885 5034525.582)
3,1,3,0,POINT (514798.300 5034522.020)
4,1,4,0,POINT (514776.531 5034562.392)
...,...,...,...,...
9446270,27890,258101,0,POINT (514785.041 5034554.476)
9446271,27890,258102,0,POINT (514837.683 5034486.200)
9446272,27890,258103,0,POINT (514891.908 5034508.959)
9446273,27890,258104,0,POINT (514865.191 5034486.992)


In [ ]:
# in un secondo abbiamo 15 frames
# dunque avremo dovremo dividere un secondo in circa 15 parti, ognuna delle quali corrisponde ad un frame

START_DATETIME = '2021-07-15 14:59:31.0'  # <----------------- da cambiare a seconda del video
NUM_PERIODS = geo_dets['frame'].max()

time_data = pd.date_range(START_DATETIME, periods = NUM_PERIODS, freq='70ms') 

list2020 = []
for i in time_data:
  list2020.append(i.strftime('%Y-%m-%d %H:%M:%S.%f'))
list2020
len(list2020)

# otteniamo una lista di orari di numero pari al numero di frame

27890

In [ ]:
x = [*range(1, geo_dets['frame'].max()+1, 1)]
len(x)

27890

In [ ]:
df_time = pd.DataFrame()
df_time['time'] = list2020
df_time['frame'] = [*range(1, geo_dets['frame'].max()+1, 1)]  # per video di 30 min
df_time.tail()

# il campo frame deve fare join con il campo frame del dataset df che abbiamo calcolato con questo script in modo da associare ad ogni frame il suo orario specifico

,time,frame
27885,2021-07-15 15:32:02.950000,27886
27886,2021-07-15 15:32:03.020000,27887
27887,2021-07-15 15:32:03.090000,27888
27888,2021-07-15 15:32:03.160000,27889
27889,2021-07-15 15:32:03.230000,27890


In [ ]:
# questo è il dataset finale
inner_merged_total = pd.merge(geo_dets, df_time, on=["frame"])
inner_merged_total.head()

,frame,id,class,geometry,time
0,1,0,0,POINT (514782.468 5034508.365),2021-07-15 14:59:31.000000
1,1,1,0,POINT (514780.687 5034524.989),2021-07-15 14:59:31.000000
2,1,2,0,POINT (514780.885 5034525.582),2021-07-15 14:59:31.000000
3,1,3,0,POINT (514798.300 5034522.020),2021-07-15 14:59:31.000000
4,1,4,0,POINT (514776.531 5034562.392),2021-07-15 14:59:31.000000


In [ ]:
inner_merged_total.tail()

,frame,id,class,geometry,time
9446270,27890,258101,0,POINT (514785.041 5034554.476),2021-07-15 15:32:03.230000
9446271,27890,258102,0,POINT (514837.683 5034486.200),2021-07-15 15:32:03.230000
9446272,27890,258103,0,POINT (514891.908 5034508.959),2021-07-15 15:32:03.230000
9446273,27890,258104,0,POINT (514865.191 5034486.992),2021-07-15 15:32:03.230000
9446274,27890,258105,0,POINT (514830.954 5034546.956),2021-07-15 15:32:03.230000


In [ ]:
# salvataggio dataset finale
inner_merged_total.to_file('detection_milan_2021-07-15_14_59_31_hour.geojson', driver='GeoJSON')

In [ ]:
!cp /content/detection_milan_2021-07-15_14_59_31_hour.geojson /content/drive/MyDrive

In [ ]:
df_geopandas = gpd.read_file('/content/drive/MyDrive/detections_YOLOv7_SORT/milan_2021-07-15_14_59_31/Copia di detection_milan_2021-07-15_14_59_31_hour.geojson')

# sarebbe più corretto chiamarli x e y
df_geopandas['lon'] = df_geopandas.geometry.x
df_geopandas['lat'] = df_geopandas.geometry.y

gdf = gpd.GeoDataFrame(df_geopandas, geometry=gpd.points_from_xy(df_geopandas.lon, df_geopandas.lat))
gdf.to_feather('detection_milan_2021-07-15_14_59_31_hour.feather')  
!cp /content/detection_milan_2021-07-15_14_59_31_hour.feather /content/drive/MyDrive